In [1]:
import pandas as pd 
import pickle 
import numpy as np 
from sqlalchemy import create_engine
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

Load Dataset

In [2]:
df = pd.read_pickle('merged.pkl')

In [3]:
df.dtypes

camis                      int64
dba                       object
boro                      object
building                  object
street                    object
zipcode                   object
inspection_date           object
critical_flag             object
record_date               object
latitude                 float64
longitude                float64
cuisine_description       object
action                    object
violation_code            object
violation_description     object
score                    float64
inspection_type           object
grade                     object
grade_date                object
address                   object
clean address             object
latitude_clean           float64
longitude_clean          float64
zipcode_clean             object
dtype: object

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228770 entries, 0 to 275867
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   camis                  228770 non-null  int64  
 1   dba                    228770 non-null  object 
 2   boro                   228770 non-null  object 
 3   building               228463 non-null  object 
 4   street                 228770 non-null  object 
 5   zipcode                228770 non-null  object 
 6   inspection_date        228770 non-null  object 
 7   critical_flag          228770 non-null  object 
 8   record_date            228770 non-null  object 
 9   latitude               228770 non-null  float64
 10  longitude              228770 non-null  float64
 11  cuisine_description    228770 non-null  object 
 12  action                 228770 non-null  object 
 13  violation_code         226980 non-null  object 
 14  violation_description  226980 non-nu

In [5]:
#drop columns not needed 
df.drop(['clean address', 'latitude_clean', 'longitude_clean', 'zipcode_clean'], axis = 1, inplace = True)

Impute Missing Address with Street name 

In [6]:
x = df[df.address.isna() & df.street.notnull()]
x

,camis,dba,boro,building,street,zipcode,inspection_date,critical_flag,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,inspection_type,grade,grade_date,address
1040,50001249,"TIGIN IRISH PUB, PEETS COFFEE&TEA/CANAL STREET...",Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2017-02-22T00:00:00.000,Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,American,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,13.0,Cycle Inspection / Initial Inspection,A,2017-02-22T00:00:00.000,NaN
1947,50008956,QDOBA MEXICAN GRILL,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2016-11-16T00:00:00.000,Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,Mexican,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,21.0,Cycle Inspection / Re-inspection,B,2016-11-16T00:00:00.000,NaN
4726,50000413,UPTOWN BRASSERIE,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2018-08-16T00:00:00.000,Not Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,American,Violations were cited in the following area(s).,22A,Nuisance created or allowed to exist. Facilit...,NaN,Administrative Miscellaneous / Initial Inspection,NaN,NaN,NaN
5236,41362369,FOOD HALL KITCHEN,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2020-02-10T00:00:00.000,Not Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,American,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,6.0,Cycle Inspection / Re-inspection,A,2020-02-10T00:00:00.000,NaN
7262,50062391,EMIRATES LOUNGE,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2019-05-30T00:00:00.000,Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,Middle Eastern,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140º F.,12.0,Cycle Inspection / Initial Inspection,A,2019-05-30T00:00:00.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272848,50086798,JOE COFFEE,Manhattan,NaN,GRAND CENTRAL TERMINAL,10017.0,2021-08-10T00:00:00.000,Critical,2022-08-27T06:00:12.000,40.752486,-73.977269,Coffee/Tea,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,13.0,Cycle Inspection / Initial Inspection,A,2021-08-10T00:00:00.000,NaN
272887,50001777,AU BON PAIN,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2017-11-01T00:00:00.000,Not Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,Soups/Salads/Sandwiches,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,12.0,Cycle Inspection / Initial Inspection,A,2017-11-01T00:00:00.000,NaN
274399,50001777,AU BON PAIN,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2017-02-23T00:00:00.000,Not Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,Soups/Salads/Sandwiches,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,25.0,Cycle Inspection / Initial Inspection,NaN,NaN,NaN
275675,41258883,O'NEALS,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,2022-06-01T00:00:00.000,Not Critical,2022-08-27T06:00:12.000,40.648313,-73.788281,American,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,4.0,Cycle Inspection / Initial Inspection,A,2022-06-01T00:00:00.000,NaN


In [7]:
#replace missing address with street name (JFK International Airport, Central Terminal)
mask1 = df['address'].isna()

mask2 = df.street.notnull()

df['address'] = np.where(mask1 & mask2, df['street'], df['address'])


In [8]:
#check if method worked
x = df[df.address.isna() & df.street.notnull()]
x

,camis,dba,boro,building,street,zipcode,inspection_date,critical_flag,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,inspection_type,grade,grade_date,address


Convert Inspection Date to Datetime 

In [9]:
df.inspection_date.value_counts()

2019-10-30T00:00:00.000    640
2019-11-07T00:00:00.000    631
2022-05-12T00:00:00.000    621
2019-10-24T00:00:00.000    616
2020-01-22T00:00:00.000    608
                          ... 
2021-05-12T00:00:00.000      1
2018-07-22T00:00:00.000      1
2016-08-09T00:00:00.000      1
2022-04-09T00:00:00.000      1
2016-07-30T00:00:00.000      1
Name: inspection_date, Length: 1618, dtype: int64

In [10]:
df.inspection_date.str.contains('T00:00:00.000').sum()

228770

In [11]:
df.inspection_date = df.inspection_date.str.replace('T00:00:00.000', '')

In [12]:
df['inspection_date'] = pd.to_datetime(df['inspection_date'], format='%Y-%m-%d')

In [13]:
df.dtypes

camis                             int64
dba                              object
boro                             object
building                         object
street                           object
zipcode                          object
inspection_date          datetime64[ns]
critical_flag                    object
record_date                      object
latitude                        float64
longitude                       float64
cuisine_description              object
action                           object
violation_code                   object
violation_description            object
score                           float64
inspection_type                  object
grade                            object
grade_date                       object
address                          object
dtype: object

In [14]:
df.inspection_date.unique()

array(['2017-08-25T00:00:00.000000000', '2022-07-07T00:00:00.000000000',
       '2022-04-11T00:00:00.000000000', ...,
       '2021-10-03T00:00:00.000000000', '2016-03-07T00:00:00.000000000',
       '2016-07-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [15]:
df.inspection_date.value_counts()

2019-10-30    640
2019-11-07    631
2022-05-12    621
2019-10-24    616
2020-01-22    608
             ... 
2021-05-12      1
2018-07-22      1
2016-08-09      1
2022-04-09      1
2016-07-30      1
Name: inspection_date, Length: 1618, dtype: int64

In [16]:
# for selecting month-year range for app 
a= df.inspection_date.drop_duplicates().sort_values()


a

13780    2015-01-06
169643   2015-06-25
162873   2015-07-08
120177   2015-09-24
119039   2015-10-14
            ...    
16003    2022-08-20
459      2022-08-22
3191     2022-08-23
356      2022-08-24
809      2022-08-25
Name: inspection_date, Length: 1618, dtype: datetime64[ns]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228770 entries, 0 to 275867
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   camis                  228770 non-null  int64         
 1   dba                    228770 non-null  object        
 2   boro                   228770 non-null  object        
 3   building               228463 non-null  object        
 4   street                 228770 non-null  object        
 5   zipcode                228770 non-null  object        
 6   inspection_date        228770 non-null  datetime64[ns]
 7   critical_flag          228770 non-null  object        
 8   record_date            228770 non-null  object        
 9   latitude               228770 non-null  float64       
 10  longitude              228770 non-null  float64       
 11  cuisine_description    228770 non-null  object        
 12  action                 228770 non-null  obje

Check for Duplicates & drop duplicates

In [18]:
# sort values by critical flag so that score 
#reflects for restaurants with critical + non critical violations on the same day as critical 
#when dropping duplicates

df = df.sort_values(['critical_flag'])

In [19]:
#check for duplicates on given datetime

(df.groupby(['camis', 'dba', 'inspection_date', 'cuisine_description'])
 .score.count().reset_index().sort_values('score', ascending = False))

,camis,dba,inspection_date,cuisine_description,score
67813,50105938,SUKI,2021-11-30,Japanese,18
68192,50106646,Secret Paradise Grill Inc,2022-08-22,Creole,16
61304,50089761,15 FLAVORS,2022-03-24,Frozen Desserts,16
45215,50056824,HUNDRED TASTE,2022-07-20,Chinese,16
38957,50042658,FISH & SHRIMP RESTAURANT SPOT,2022-08-02,Latin American,15
...,...,...,...,...,...
25483,41683539,HAPPY GARDEN CHINESE RESTAURANT,2018-12-17,Chinese,0
54787,50076107,LUCKY,2021-06-16,Other,0
25442,41682958,EL ANCLA DE ASTORIA RESTAURANT & BAR,2016-11-03,Peruvian,0
54768,50076079,CHINITO LATINO,2019-03-12,Chinese/Cuban,0


In [20]:
mask = ((df['camis'] ==50105938) &
       (df['dba'] =="SUKI") &
       (df['inspection_date'].dt.date == datetime.datetime(2021,11,30).date()) &
       (df['cuisine_description'] =="Japanese")
       
)
df[mask]

,camis,dba,boro,building,street,zipcode,inspection_date,critical_flag,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,inspection_type,grade,grade_date,address
176319,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,06E,"Sanitized equipment or utensil, including in-u...",105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
173093,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,02B,Hot food item not held at or above 140º F.,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
130762,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,04H,"Raw, cooked or prepared food is adulterated, c...",105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
232498,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,04N,Filth flies or food/refuse/sewage-associated (...,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
271960,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,04M,Live roaches present in facility's food and/or...,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
231347,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,04L,Evidence of mice or live mice present in facil...,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
227489,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,06D,"Food contact surface not properly washed, rins...",105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
44237,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,05D,Hand washing facility not provided in or near ...,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
68855,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,04K,Evidence of rats or live rats present in facil...,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC
63111,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,04A,Food Protection Certificate not held by superv...,105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC


In [21]:
#get rid of duplicates
df.drop_duplicates(subset =['camis', 'dba', 'inspection_date', 'cuisine_description'], inplace =True)

In [22]:
#check if duplicates are gone
#check for duplicates on given datetime

(df.groupby(['camis', 'dba', 'inspection_date', 'cuisine_description'])
 .score.count().reset_index().sort_values('score', ascending = False))

,camis,dba,inspection_date,cuisine_description,score
0,30075445,Test test Test test Test test Test test Test t...,2019-05-16,Bakery Products/Desserts,1
48282,50063620,DIAMOND LIL,2017-10-14,American,1
48288,50063627,JOE & THE JUICE,2019-07-01,"Juice, Smoothies, Fruit Salads",1
48287,50063625,UTOPIA DINER,2022-05-02,American,1
48286,50063625,UTOPIA DINER,2020-02-27,American,1
...,...,...,...,...,...
17599,41450971,GIOVANNI'S RESTAURANT,2018-12-20,Pizza,0
69471,50109601,INSOMNIA COOKIES,2022-07-13,American,0
46932,50059875,1ST MAMA RESTAURANT,2020-01-09,Latin American,0
57128,50081412,GAZALA'S,2022-05-19,Mediterranean,0


In [23]:
mask = ((df['camis'] ==50105938) &
       (df['dba'] =="SUKI") &
       (df['inspection_date'].dt.date == datetime.datetime(2021,11,30).date()) &
       (df['cuisine_description'] =="Japanese")
       
)
df[mask]

,camis,dba,boro,building,street,zipcode,inspection_date,critical_flag,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,inspection_type,grade,grade_date,address
176319,50105938,SUKI,Manhattan,111,1 AVENUE,10003.0,2021-11-30,Critical,2022-08-27T06:00:12.000,40.726827,-73.985853,Japanese,Establishment Closed by DOHMH. Violations were...,06E,"Sanitized equipment or utensil, including in-u...",105.0,Pre-permit (Operational) / Initial Inspection,NaN,NaN,111 1 AVENUE NYC


In [24]:
df.shape

(72641, 20)

Impute missing values for Grade Category

In [25]:
df.grade.value_counts()

A    39195
B     3002
N     1602
C     1026
Z      861
P      610
Name: grade, dtype: int64

In [26]:
df.grade.unique()

array([nan, 'A', 'C', 'N', 'B', 'Z', 'P'], dtype=object)

In [27]:
df['grade'] = df['grade'].replace(np.nan, 'Missing Grade')

In [28]:
df['grade'] = df['grade'].replace('N', 'Not Yet Graded')
df['grade'] = df['grade'].replace('Z', 'Grade Pending')
df['grade'] = df['grade'].replace('P', 'Grade Pending resulted in a closure')


In [29]:
df.grade.value_counts()

A                                      39195
Missing Grade                          26345
B                                       3002
Not Yet Graded                          1602
C                                       1026
Grade Pending                            861
Grade Pending resulted in a closure      610
Name: grade, dtype: int64

Impute Missing Score values w/ median zipcode score of matched zipcode 

In [30]:
zipcode_map = (df.query('score.notnull()')
              .drop_duplicates('zipcode')
              .set_index('zipcode')
              .score)

zipcode_map

zipcode
11225.0    24.0
10475.0    77.0
10012.0     7.0
11209.0     9.0
10465.0    34.0
           ... 
11214       7.0
11105       0.0
11241.0     5.0
10199      12.0
10470       2.0
Name: score, Length: 325, dtype: float64

In [31]:
df['score'] = df['zipcode'].map(zipcode_map).fillna(np.nan)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72641 entries, 0 to 186097
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   camis                  72641 non-null  int64         
 1   dba                    72641 non-null  object        
 2   boro                   72641 non-null  object        
 3   building               72511 non-null  object        
 4   street                 72641 non-null  object        
 5   zipcode                72641 non-null  object        
 6   inspection_date        72641 non-null  datetime64[ns]
 7   critical_flag          72641 non-null  object        
 8   record_date            72641 non-null  object        
 9   latitude               72641 non-null  float64       
 10  longitude              72641 non-null  float64       
 11  cuisine_description    72641 non-null  object        
 12  action                 72641 non-null  object        
 13  

In [33]:
#save to pickple
df.to_pickle('final_clean.pkl')